### Import Required Libraries and Set Up Environment Variables

In [88]:
# Dependencies
from urllib.parse import urlencode
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import time
import os

In [89]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

print("NTY API Loaded:", type(nyt_api_key))
print("Movie DB API Loaded:", type(tmdb_api_key))

NTY API Loaded: <class 'str'>
Movie DB API Loaded: <class 'str'>


### Access the New York Times API

In [90]:
# Set the base URL
url = "https://api.nytimes.com"
endpoint = "/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

query_params = {
    "begin_date": begin_date,
    "end_date": end_date,
    "fl": field_list,
    "fq": filter_query,
    "sort": sort,
    "api-key": nyt_api_key
}

encoded_params = urlencode(query_params)

# Build URL
full_url = (
    f"{url}"
    f"{endpoint}"
    f"{encoded_params}"
)

# Print URL (Hiding API Key)
print("Full URL:", full_url.replace(nyt_api_key, "<API_KEY>"))

Full URL: https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20130101&end_date=20230531&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&fq=section_name%3A%22Movies%22+AND+type_of_material%3A%22Review%22+AND+headline%3A%22love%22&sort=newest&api-key=<API_KEY>


In [91]:
# Create an empty list to store the reviews
reviews_list = []

#! loop through pages 0-19
for page_index in range(0, 3):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{full_url}&page={page_index}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews_data = response.json()

    #! Add a twelve second interval between queries to stay within API query limits
    time.sleep(0.1)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews_data["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Page {page_index} retrieved successfully!")

    except KeyError as error:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page_index}.", f"Error: {error}")

Page 0 retrieved successfully!
Page 1 retrieved successfully!
Page 2 retrieved successfully!


In [92]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [93]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

print("Shape:", reviews_df.shape)
reviews_df

Shape: (30, 16)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}]",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Kapur, Shekhar', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'James, Lily', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Azmi, Shabana', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'Thompson, Emma', 'rank': 5, 'major': 'N'}]",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'You Can Live Forever (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Slutsky, Mark', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Watts, Sarah (Film Director)', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'O'Driscoll, Anwen', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Laporte, June (Actor)', 'rank': 6, 'major': 'N'}]",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'A Tourist's Guide to Love (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Tsuchida, Steven', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Cook, Rachael Leigh', 'rank': 4, 'major': 'N'}]",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New Yo

In [94]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  30 non-null     object
 1   snippet                  30 non-null     object
 2   source                   30 non-null     object
 3   keywords                 30 non-null     object
 4   pub_date                 30 non-null     object
 5   word_count               30 non-null     int64 
 6   headline.main            30 non-null     object
 7   headline.kicker          6 non-null      object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  30 non-null     object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null      object
 13  byline.original          30 non-null     object
 14  byline.person            30 non-null     obj

In [95]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

print("Shape:", reviews_df.shape)
reviews_df

Shape: (30, 17)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}]",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Kapur, Shekhar', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'James, Lily', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Azmi, Shabana', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'Thompson, Emma', 'rank': 5, 'major': 'N'}]",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'You Can Live Forever (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Slutsky, Mark', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Watts, Sarah (Film Director)', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'O'Driscoll, Anwen', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Laporte, June (Actor)', 'rank': 6, 'major': 'N'}]",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'A Tourist's Guide to Love (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Tsuchida, Steven', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Cook, Rachael Leigh', 'rank': 4, 'major': 'N'}]",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as

In [96]:
reviews_df['title'].values

array(['The Attachment Diaries',
       'What’s Love Got to Do With It?’ Probably a Lo',
       'You Can Live Forever', 'A Tourist’s Guide to Love',
       'Other People’s Children', 'One True Loves',
       'The Lost Weekend: A Love Story', 'A Thousand and One',
       'Your Place or Mine', 'Love in the Time of Fentanyl',
       'Pamela, a Love Story', 'In From the Side', 'After Love',
       'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover',
       'The Sound of Christmas', 'The Inspection', 'Bones and All',
       'My Policeman', 'About Fate', 'Waiting for Bojangles',
       'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love',
       'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade',
       'Fire of Love'], dtype=object)

In [97]:
reviews_df['headline.main'].values

array(['‘The Attachment Diaries’ Review: Love, Sick',
       'Review: ‘What’s Love Got to Do With It?’ Probably a Lot',
       '‘You Can Live Forever’ Review: Do You Love Me Now?',
       '‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip',
       '‘Other People’s Children’ Review: True Romance',
       '‘One True Loves’ Review: A Romance Lost at Sea',
       '‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed',
       '‘A Thousand and One’ Review: A New York Love Story',
       '‘Your Place or Mine’ Review: Try Neither',
       '‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouver',
       '‘Pamela, a Love Story’ Review: A Frank Look Back',
       '‘In From the Side’ Review: Love and Rugby Play a Losing Game',
       '‘After Love’ Review: The Other Woman',
       '‘Alcarràs’ Review: Labor of Love',
       '‘Nelly & Nadine’ Review: An Unlikely Love, an Unlikely Record',
       '‘Lady Chatterley’s Lover’ Review: When Connie Met Ollie

In [98]:
reviews_df['keywords'].values[0]

[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
 {'name': 'creative_works',
  'value': 'The Attachment Diaries (Movie)',
  'rank': 2,
  'major': 'N'},
 {'name': 'persons',
  'value': 'Diment, Valentin Javier',
  'rank': 3,
  'major': 'N'}]

In [99]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df["keywords"].apply(extract_keywords)

In [100]:
reviews_df['keywords'].values[0]

'subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;'

In [101]:
print("Shape:", reviews_df.shape)
reviews_df

Shape: (30, 17)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins

In [102]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()

titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love']

### Access The Movie Database API

In [103]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org"
search_endpoint = "/3/search/movie"

In [116]:
# Create an empty list to store the results
tmdb_movies_list =[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter =1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print(f"Number of requests reached {request_counter}. Sleeping for a sec...")

    # Add 1 to the request counter
    request_counter += 1

    query_params = {
        "query": title,
        "api_key": tmdb_api_key
    }

    encoded_params = urlencode(query_params)
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(f"{url}{search_endpoint}?{encoded_params}")
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = data["results"][0]["id"]

        # Make a request for a the full movie details
        movie_endpoint = f"/3/movie/{movie_id}"
        query_url = f"{url}{movie_endpoint}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        data = requests.get(query_url).json()


        
        # Extract the genre names into a list
        genres = [genre["name"] for genre in data["genres"]]     

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language["english_name"] for language in data["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in data["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(
            {
                "title": data['title'],
                "original_title": data['original_title'],
                "budget": data['budget'],
                "genre": genres,
                "language": data['original_language'],
                "spoken_languages": spoken_languages,
                "homepage": data['homepage'],
                "overview": data['overview'],
                "popularity": data['popularity'],
                "runtime": data['runtime'],
                "revenue": data['revenue'],
                "release_date": data['release_date'],
                "vote_average": data['vote_average'],
                "vote_count": data['vote_count'],
                "production_countries": production_countries
            }
        )
        
        # Print out the title that was found
        print(f"Found '{title}'.")
        
    except IndexError:
        print(f"'{title}' not found.")



Found 'The Attachment Diaries'.
'What’s Love Got to Do With It?’ Probably a Lo' not found.
Found 'You Can Live Forever'.
Found 'A Tourist’s Guide to Love'.
Found 'Other People’s Children'.
Found 'One True Loves'.
Found 'The Lost Weekend: A Love Story'.
Found 'A Thousand and One'.
Found 'Your Place or Mine'.
Found 'Love in the Time of Fentanyl'.
Found 'Pamela, a Love Story'.
Found 'In From the Side'.
Found 'After Love'.
Found 'Alcarràs'.
Found 'Nelly & Nadine'.
Found 'Lady Chatterley’s Lover'.
Found 'The Sound of Christmas'.
Found 'The Inspection'.
Found 'Bones and All'.
Found 'My Policeman'.
Found 'About Fate'.
Found 'Waiting for Bojangles'.
Found 'I Love My Dad'.
Found 'A Love Song'.
Found 'Alone Together'.
Found 'Art of Love'.
Found 'The Wheel'.
Found 'Thor: Love and Thunder'.
Found 'Both Sides of the Blade'.
Found 'Fire of Love'.


In [105]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [106]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [107]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [108]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [109]:
# Drop "byline.person" column


In [110]:
# Delete duplicate rows and reset index


In [111]:
# Export data to CSV without the index
